带硬算子的socket脚本   
区别在于，配置硬算子后，输出layout发生了变化，需要根据硬件输出的layout进行数据排布的转化、后处理，整体流程与不带硬算子的保持一致。   
下面，让我们开始部署网络吧！

首先import相关的包，分为三部分， 框架下相关包，icraft相关包，模型的前后处理、可视化   
注意这里的后处理，已经是对应硬算子的后处理（detpost_hard）了，请注意区分。
* xir:Icraft-XIR的python模块
* xrt:Icraft-XIR的python模块(runtime运行时)
* host_backend:Icraft-HostBackend的python模块
* buyibackend:Icraft-BuyiBackend的python模块
* preprocess:前处理模块
* utils:功能函数模块，主要用来获取量化参数scale_list
* detpost_hard:配硬算子的后处理模块
* visualize:结果可视化模块

In [1]:
import torch
import numpy as np
from typing import List
import cv2
import json
import math 

from icraft.xir import *
from icraft.xrt import *
from icraft.host_backend import *
from icraft.buyibackend import *

from preprocess import letterbox,scale_coords
from utils import dmaInit,fpgaOPlist,getOutputNormratio
from detpost_hard import soft_nms,get_det_results
from visualize import vis,COCO_CLASSES

声明一些文件路径，主要是板子ip地址、icraft编译后的指令生成网络（json&raw文件）、测试图片

In [2]:
# ---------------------------------参数设置---------------------------------
# 路径设置 
URL_PATH =  R"socket://ql100aiu@192.168.125.184:9981?npu=0x40000000&dma=0x80000000"
GENERATED_JSON_FILE = "./imodel/yolov5s_hard/yolov5s_BY.json"
GENERATED_RAW_FILE = "./imodel/yolov5s_hard/yolov5s_BY.raw"
IMG_PATH="./images/000000000885.jpg" 

下面是最主要的模型前向过程，分为以下几个步骤   
* 1.开启device
* 2.加载指令生成后的网络
* 3.创建session并将整个网络绑定到BuyiBackend和HostBackend，使用Buyi_device和HostDevice
* 4.加载测试图像并转成icraft.Tensor
* 5.配置IMK，将ps端数据搬移至udma buffer （这一步不要忘记哦，否则会引起错误哦）
* 6.模型前向推理

In [3]:
# open device
device = Device.Open(URL_PATH)
print('INFO:open device at:',URL_PATH) 

INFO:open device at: socket://ql100aiu@192.168.125.184:9981?npu=0x40000000&dma=0x80000000


In [4]:
# 加载指令生成后的网络
network = Network.CreateFromJsonFile(GENERATED_JSON_FILE)
network.loadParamsFromFile(GENERATED_RAW_FILE)
print('INFO: Create network!')

INFO: Create network!


In [5]:
# 创建session并应用
session = Session.Create([ BuyiBackend, HostBackend], network.view(1), [ device,HostDevice.Default()])
session.apply()

In [6]:
# 加载测试图像并转成icraft.Tensor
img=cv2.imread(IMG_PATH)
ri = letterbox(img,new_shape=(640,640),stride=32,auto=False)[0]
img_=ri.reshape(1,640,640,3)
input_tensor = Tensor(img_, Layout("NHWC"))
print('INFO: load test image!')

INFO: load test image!


In [7]:
# dma init
customop_set=fpgaOPlist(network)
print(customop_set)
IMK = True if "customop::ImageMakeNode" in customop_set else False
print('IMK =',IMK) #获取网络中是否使用了IMK硬算子
dmaInit(device,input_tensor,[640,640,3],IMK) #如果使用了IMK硬算子，进行对应配置
print('dmaInit done')

{'customop::DetPostNode', 'customop::ImageMakeNode'}
IMK = True
dmaInit done


In [8]:
# 模型前向推理
output_tensors = session.forward( [input_tensor] )
# 养成良好的使用习惯，网络前向结束后，重置device
device.reset(0)

此时，我们就获得了模型前向推理的结果，可以看一下三个检测头的输出结果！
由于结果还在udma buffer中，需要调用tensor.to方法将输出结果从udma buffer搬运至ps端进行后续处理，这一步不要忘记哦
请关注tensor.to(rt.HostDevice.MemRegion())的使用

In [9]:
generated_output = []
for tensor in output_tensors:
    generated_output.append(np.asarray(tensor.to(HostDevice.MemRegion())))#data transfer2 [udma->ps]  

print('anchor_0 shape =',np.array(generated_output[0]).shape) # [1,1,152,96]
print('anchor_1 shape =',np.array(generated_output[1]).shape) # [1,1,100,96]
print('anchor_2 shape =',np.array(generated_output[2]).shape) # [1,1,56,96]
print('INFO: get icore results!')

anchor_0 shape = (1, 1, 760, 96)
anchor_1 shape = (1, 1, 340, 96)
anchor_2 shape = (1, 1, 106, 96)
INFO: get icore results!


可以看到，输出的数据排布与不配硬算子时已近发生了变化，接下来要针对硬件输出的layout进行数据排布的转换，将icore_post的结果转化为检测结果。
后处理一共包含3步，分别是：   
* Step 1: 读取norm_ratio，对定点结果进行反量化
* Step 2: 将icore_post结果转化为box_list,scores_list,id_list
* Step 3: 进行非极大值抑制，取NMS后的结果

In [10]:
 # Step1: 因为icore_post输出的是定点结果，所以需要读取norm_ratio，对定点结果进行反量化
scale_list = getOutputNormratio(network)
print('INFO: get scale results!')  
print(scale_list)

INFO: get scale results!
[0.1493464342252476, 0.1250864539559432, 0.11689911489411602]


Step 2: 将icore_post结果转化为box_list,scores_list,id_list
主要是根据硬件的数据排布格式，从[x,96]维度的icore_post硬件输出中计算x,y,w,h坐标   

Step 1中提取的量化scale也在本步骤中对定点结果进行反量化，因此输出已经是浮点结果。
有兴趣的可自行查看封装好的get_det_results和soft_nms函数内部实现过程。

In [11]:
# Step 2: 将icore_post结果转化为box_list,scores_list,id_list
scores_list,box_list,id_list = get_det_results(generated_output,scale_list)
print('INFO: get detection results!') 

INFO: get icore_post flatten results!
INFO: get detection results!


In [12]:
# Step 3: NMS 取NMS后的结果
nms_indices,nms_box_list,nms_score_list,nms_cls_ids = soft_nms(box_list, scores_list, id_list)
print('nms_indices =',nms_indices)
print('INFO: get NMS results!')  

nms_indices = [1181, 1122, 1163, 640, 681, 1150]
INFO: get NMS results!


最终，将结果进行可视化，就可以看到配置硬算子的上板结果啦~是不是很简便，快来尝试着部署自己的网络吧:)

In [13]:
# ---------------------------------结果可视化---------------------------------
nms_box_list = scale_coords(img_.shape[1:], np.array(nms_box_list), img.shape)
result_image = vis(img,boxes=nms_box_list,scores=nms_score_list,cls_ids=nms_cls_ids,conf=0.25,class_names=COCO_CLASSES)
cv2.imshow(" ", result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

如果需要继续测试其他图片，只需要加载新的测试图片、重置device、重新调用session.forward即可。
建议养成良好的使用习惯，每次session.forward后重置device，确保连续测试不同图像的正确性。

In [14]:
IMG1_PATH="./images/000000000139.jpg" 
# 加载测试图像并转成icraft.Tensor
img=cv2.imread(IMG1_PATH)
ri = letterbox(img,new_shape=(640,640),stride=32,auto=False)[0]
img_=ri.reshape(1,640,640,3)
input_tensor = Tensor(img_, Layout("NHWC"))
print('INFO: load test image!')

#如果使用了IMK硬算子，进行对应配置
dmaInit(device,input_tensor,[640,640,3],IMK) 
print('dmaInit done')
# 模型前向推理
output_tensors = session.forward( [input_tensor] )
# 重置device   这一步很重要！！！重置device   这一步很重要！！！重置device   这一步很重要！！！
device.reset(1)
# 将网络前向结果从PL搬到PS
generated_output = []
for tensor in output_tensors:
    generated_output.append(np.asarray(tensor.to(HostDevice.MemRegion())))#data transfer2 [udma->ps]  

print('anchor_0 shape =',np.array(generated_output[0]).shape) # [1,1,152,96]
print('anchor_1 shape =',np.array(generated_output[1]).shape) # [1,1,100,96]
print('anchor_2 shape =',np.array(generated_output[2]).shape) # [1,1,56,96]
print('INFO: get icore results!')
 # Step1: 因为icore_post输出的是定点结果，所以需要读取norm_ratio，对定点结果进行反量化
scale_list = getOutputNormratio(network)
print('INFO: get scale results!')  
print(scale_list)
# Step 2: 将icore_post结果转化为box_list,scores_list,id_list
scores_list,box_list,id_list = get_det_results(generated_output,scale_list)
print('INFO: get detection results!') 
# Step 3: NMS 取NMS后的结果
nms_indices,nms_box_list,nms_score_list,nms_cls_ids = soft_nms(box_list, scores_list, id_list)
print('nms_indices =',nms_indices)
print('INFO: get NMS results!')  
# ---------------------------------结果可视化---------------------------------
nms_box_list = scale_coords(img_.shape[1:], np.array(nms_box_list), img.shape)
result_image = vis(img,boxes=nms_box_list,scores=nms_score_list,cls_ids=nms_cls_ids,conf=0.25,class_names=COCO_CLASSES)
cv2.imshow(" ", result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

INFO: load test image!
dmaInit done
anchor_0 shape = (1, 1, 1382, 96)
anchor_1 shape = (1, 1, 683, 96)
anchor_2 shape = (1, 1, 220, 96)
INFO: get icore results!
INFO: get scale results!
[0.1493464342252476, 0.1250864539559432, 0.11689911489411602]
INFO: get icore_post flatten results!
INFO: get detection results!
nms_indices = [1457, 1890, 2145, 1747, 1981, 1983, 1625, 931, 2049, 2167, 2122, 2189, 1461, 335, 1108, 137, 516]
INFO: get NMS results!


部署完毕，不要忘记关闭设备哦~

In [15]:
# 关闭设备
Device.Close(device)